In [1]:
import os
import sys
import random
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
class MLP:
    def __init__(self, vocab_size, hidden_size):
        self._vocab_size = vocab_size
        self._hidden_size = hidden_size
    
    def build_graph(self):
        self._X = tf.placeholder(tf.float32, shape=[None, self._vocab_size])
        self._real_Y = tf.placeholder(tf.int32, shape=[None, ])
        
        weights_1 = tf.get_variable(
            name = 'weight_input_hidden',
            shape = (self._vocab_size, self._hidden_size),
            initializer = tf.random_normal_initializer(seed=2022),
        )
        biases_1 = tf.get_variable(
            name = 'biases_input_hidden',
            shape = (self._hidden_size),
            initializer = tf.random_normal_initializer(seed=2022),
        )
        weights_2 = tf.get_variable(
            name = "weights_hidden_output",
            shape = (self._hidden_size, NUM_CLASSES),
            initializer = tf.random_normal_initializer(seed=2022),
        )
        biases_2 = tf.get_variable(
            name = "biases_hidden_output",
            shape = (NUM_CLASSES),
            initializer = tf.random_normal_initializer(seed=2022),
        )
        hidden = tf.matmul(self._X, weights_1) + biases_1
        hidden = tf.sigmoid(hidden)
        logits = tf.matmul(hidden, weights_2) + biases_2
        
        labels_one_hot = tf.one_hot(indices=self._real_Y, depth=NUM_CLASSES, dtype=tf.float32)
        loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels_one_hot, logits=logits)
        loss = tf.reduce_mean(loss)
        probs = tf.nn.softmax(logits)
        predicted_labels = tf.argmax(probs, axis=1)
        predicted_labels = tf.squeeze(predicted_labels)
        
        return predicted_labels, loss
    
    def trainer(self, loss, learning_rate):
        train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
        return train_op

In [3]:
class DataReader:
    def __init__(self, data_path, batch_size, vocab_size):
        self._batch_size = batch_size
        with open(data_path) as f:
            d_lines = f.read().splitlines()

        self._data = []
        self._labels = []
        for line in d_lines:
            vector = [0.0 for _ in range(vocab_size)]
            features = line.split("<fff>")
            label, doc_id = int(features[0]), int(features[1])
            tokens = features[2].split()
            for token in tokens:
                index, value = int(token.split(":")[0]), float(token.split(":")[1])
                vector[index] = value
            self._data.append(vector)
            self._labels.append(label)

        self._data = np.array(self._data)
        self._labels = np.array(self._labels)
        self._num_epoch = 0
        self._batch_id = 0

    def next_batch(self):
        start = self._batch_id * self._batch_size
        end = min(start + self._batch_size, len(self._data))
        self._batch_id += 1

        if end == len(self._data):
            self._num_epoch += 1
            self._batch_id = 0
            indices = list(range(len(self._data)))
            random.seed(2022)
            random.shuffle(indices)
            self._data, self._labels = self._data[indices], self._labels[indices]
        return self._data[start:end], self._labels[start:end]

In [4]:
def save_parameters(name, value, epoch):
    filename = name.replace(":", "-colon-") + "-epoch-{}.txt".format(epoch)
    if len(value.shape) == 1:
        string_form = ",".join([str(number) for number in value])
    else:
        string_form = "\n".join(
            [
                ",".join([str(number) for number in value[row]])
                for row in range(value.shape[0])
            ]
        )
    with open(os.getcwd() + "/saved-paras/" + filename, "w") as f:
        f.write(string_form)

In [5]:
def load_dataset(vocab_size):
    train_data_reader = DataReader(
        data_path=os.getcwd() + "/20news-bydate/20news-train-tf-idf.txt",
        batch_size=50,
        vocab_size=vocab_size,
    )

    test_data_reader = DataReader(
        data_path=os.getcwd() + "/20news-bydate/20news-test-tf-idf.txt",
        batch_size=50,
        vocab_size=vocab_size,
    )

    return train_data_reader, test_data_reader

In [6]:
def restore_parameters(name, epoch):
    filename = name.replace(':', '-colon-') + '-epoch-{}.txt'.format(epoch)
    with open(os.getcwd()+"/saved-paras/"+filename) as f:
        lines = f.read().splitlines()
    if len(lines) == 1:
        value = [float(number) for number in lines[0].split(',')]
    else:
        value = [[float(number) for number in lines[row].split(',')] for row in range(len(lines))]
    return value

In [7]:
NUM_CLASSES =20

# Load features size
with open(os.getcwd() + '/20news-bydate/20news-full-words-idfs.txt') as f:
    vocab_size = len(f.read().splitlines())

# mlp initialization
mlp = MLP(vocab_size=vocab_size, hidden_size=50)

# build graph model
predicted_labels, loss = mlp.build_graph()

# optimizator
train_op = mlp.trainer(loss=loss, learning_rate=0.1)

# open a session
with tf.Session() as sess:
    train_data_reader, test_data_reader = load_dataset(vocab_size)
    
    # run session
    sess.run(tf.global_variables_initializer())

    steps = 500
    # training
    for step in range(steps):
        # load batch
        train_data, train_labels = train_data_reader.next_batch()
        
        # feeding
        labels_eval, loss_eval, _ = sess.run(
            [predicted_labels, loss, train_op],
            feed_dict={
                mlp._X: train_data,
                mlp._real_Y: train_labels
            }
        )
        
        # results
        print('step: {}, loss: {}'.format(step, loss_eval))
        
        # save params before train each epouch
        if train_data_reader._batch_id == 0:
            trainable_variables = tf.trainable_variables()
            for variable in trainable_variables:
                save_parameters(
                name = variable.name,
                value = variable.eval(),
                epoch = train_data_reader._num_epoch
            )

# load saved params
with tf.Session() as sess:
    # load final epoch params
    epoch = train_data_reader._num_epoch
    trainable_variables = tf.trainable_variables()
    for variable in trainable_variables:
        saved_value = restore_parameters(variable.name, epoch)
        assign_op = variable.assign(saved_value)
        sess.run(assign_op)

    # load one epoch to evaluate
    num_true_preds = 0
    while True:  
        test_data, test_labels = test_data_reader.next_batch()
        test_labels_eval = sess.run(
            predicted_labels,
            feed_dict = {
                mlp._X: test_data,
                mlp._real_Y: test_labels
            }
        )
        matches = np.equal(test_labels_eval, test_labels)
        num_true_preds += np.sum(matches.astype(float))
        if test_data_reader._batch_id == 0:
            break
    print('Epoch:', epoch)
    print('Accuracy on test data:', num_true_preds/len(test_data_reader._data))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

step: 0, loss: 9.414592742919922
step: 1, loss: 0.6328482031822205
step: 2, loss: 0.0005247645312920213
step: 3, loss: 2.7298485747451195e-06
step: 4, loss: 8.344546245098172e-07
step: 5, loss: 0.0
step: 6, loss: 0.0
step: 7, loss: 0.0
step: 8, loss: 0.0
step: 9, loss: 14.02230453491211
step: 10, loss: 33.304203033447266
step: 11, loss: 29.2415828704834
step: 12, loss: 24.001264572143555
step: 13, loss: 17.916475296020508
step: 14, loss: 11.774836540222168
step: 15, loss: 5.9914116859436035
step: 16, loss: 0.9426317811012268
step: 17, loss: 0.12705358862876892
step: 18, loss: 0.0004097946221008897
step: 19, loss: 3.824170107691316e-06
step: 20, loss: 1.1682499234666466e-07
step: 21, loss: 18.593366622924805
step: 22, loss: 25.262248992919922
step: 23, loss: 22.324996948242188
step: 24, loss: 19.371

step: 226, loss: 2.474700450897217
step: 227, loss: 2.760728359222412
step: 228, loss: 2.6415648460388184
step: 229, loss: 2.7318222522735596
step: 230, loss: 2.7153613567352295
step: 231, loss: 2.7346057891845703
step: 232, loss: 2.499502658843994
step: 233, loss: 2.7657365798950195
step: 234, loss: 2.756166458129883
step: 235, loss: 2.939451217651367
step: 236, loss: 2.7604901790618896
step: 237, loss: 2.7466864585876465
step: 238, loss: 2.5600271224975586
step: 239, loss: 2.766481637954712
step: 240, loss: 2.663541316986084
step: 241, loss: 2.802391290664673
step: 242, loss: 2.522230386734009
step: 243, loss: 2.617781162261963
step: 244, loss: 2.4638259410858154
step: 245, loss: 2.728048801422119
step: 246, loss: 2.51198410987854
step: 247, loss: 2.2539498805999756
step: 248, loss: 2.4254581928253174
step: 249, loss: 2.1723697185516357
step: 250, loss: 2.2241404056549072
step: 251, loss: 1.8455015420913696
step: 252, loss: 2.2493278980255127
step: 253, loss: 1.9516651630401611
step:

step: 471, loss: 0.39694905281066895
step: 472, loss: 0.27493569254875183
step: 473, loss: 0.151875302195549
step: 474, loss: 0.2205466330051422
step: 475, loss: 0.16700798273086548
step: 476, loss: 0.12803015112876892
step: 477, loss: 0.21930532157421112
step: 478, loss: 0.061292730271816254
step: 479, loss: 0.2259766161441803
step: 480, loss: 0.2562943398952484
step: 481, loss: 0.2500527501106262
step: 482, loss: 0.10544415563344955
step: 483, loss: 0.35060936212539673
step: 484, loss: 0.4285593330860138
step: 485, loss: 0.15079961717128754
step: 486, loss: 0.2637934386730194
step: 487, loss: 0.026384472846984863
step: 488, loss: 0.2554023861885071
step: 489, loss: 0.1544821858406067
step: 490, loss: 0.32247376441955566
step: 491, loss: 0.19524602591991425
step: 492, loss: 0.053897857666015625
step: 493, loss: 0.10523629188537598
step: 494, loss: 0.24517516791820526
step: 495, loss: 0.12078118324279785
step: 496, loss: 0.3094165623188019
step: 497, loss: 0.16061297059059143
step: 498